In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

#### Load Data

In [2]:
df = pd.read_csv("duopoly_competition_details copy.csv")
df = df[(df['selling_season'] == df['selling_season'][0]) & (df['competition_id'] == df['competition_id'][0])]
X = df[[ 'price','price_competitor']]
y = df[['demand']] 

X = X.to_numpy()
y = np.asarray(y).reshape(-1)
# plt.plot(X[0:], y)
print(X, y)

[[48.  46.6]
 [57.6 46.4]
 [69.1 66.9]
 [82.9 45.4]
 [94.  45. ]
 [97.2 47.7]
 [97.3 54. ]
 [93.  57.5]
 [85.7 52.7]
 [77.  47.9]
 [68.  44.2]
 [59.  62.8]
 [55.7 68.8]
 [54.  58.5]
 [53.  66.3]
 [50.8 52.2]
 [49.5 56. ]
 [49.1 47.6]
 [49.6 63.9]
 [50.9 58.8]
 [51.6 41.2]
 [51.5 41. ]
 [50.8 42.1]
 [49.1 64.3]
 [49.3 65.9]
 [49.  54.7]
 [48.1 64.5]
 [47.6 50. ]
 [46.6 68.8]
 [46.  67.1]
 [47.4 40.4]
 [48.6 55.7]
 [50.1 51.6]
 [52.1 60.8]
 [53.6 60.4]
 [55.5 60.3]
 [57.9 43.5]
 [59.9 59.9]
 [63.2 58.9]
 [66.4 42.5]
 [69.1 67.2]
 [71.3 63.2]
 [73.1 47.7]
 [74.4 50.9]
 [75.2 45.1]
 [75.5 60.4]
 [75.3 48.6]
 [74.8 55. ]
 [74.  50.1]
 [72.6 48. ]
 [70.7 57. ]
 [68.5 63.7]
 [67.3 46.2]
 [65.5 54.7]
 [63.3 65.5]
 [60.7 50.4]
 [58.  57.8]
 [55.6 49. ]
 [53.1 49.5]
 [50.3 59.1]
 [47.4 59. ]
 [45.6 44.7]
 [44.  42. ]
 [42.4 66.2]
 [40.9 59.2]
 [39.8 48. ]
 [38.7 44.7]
 [37.3 59.6]
 [36.5 41.1]
 [35.9 57.2]
 [35.9 47.9]
 [35.9 58.1]
 [35.7 62.9]
 [35.5 65.7]
 [35.6 66.4]
 [35.4 43.1]
 [35.2 43.1]

## LAD

In [3]:
import pyomo.environ as pyo
solver = 'ipopt'
SOLVER = pyo.SolverFactory(solver)
assert SOLVER.available(), f"Solver {solver} is not available."

In [4]:
def lad_regression(X, y):
    model = pyo.ConcreteModel("LAD Regression")

    # get dimensions of data
    n, k = X.shape

    # create index sets (note use of Python style zero based indexing)
    model.I = pyo.RangeSet(0, n - 1)
    model.J = pyo.RangeSet(0, k - 1)

    # create variables
    model.ep = pyo.Var(model.I, domain=pyo.NonNegativeReals)
    model.em = pyo.Var(model.I, domain=pyo.NonNegativeReals)
    model.m = pyo.Var(model.J)
    model.b = pyo.Var()

    # constraints
    @model.Constraint(model.I)
    def residuals(m, i):
        return m.ep[i] - m.em[i] == y[i] - sum(X[i][j] * m.m[j] for j in m.J) - m.b

    # objective
    @model.Objective(sense=pyo.minimize)
    def sum_of_abs_errors(m):
        return sum(m.ep[i] + m.em[i] for i in m.I)

    return model

model = lad_regression(X, y)
SOLVER.solve(model)
model.m.display()
model.b.display()
# print(
#     f"Optimal solution: m = {[f'{model.m[j].value:.5f}' for j in model.J[0]]} and b = {model.b.value:.5f}"
# )
# print(f"Objective value: {pyo.value(model.sum_of_abs_errors):.2f}")

m : Size=2, Index=J
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
      0 :  None : -0.028805673020746495 :  None : False : False :  Reals
      1 :  None :  0.019942388817977025 :  None : False : False :  Reals
b : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 1.1544427346176096 :  None : False : False :  Reals


## OLS

In [5]:
solver = 'ipopt'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)
assert SOLVER.available(), f"Solver {solver} is not available."

In [6]:
def ols_regression(X, y):
    m = pyo.ConcreteModel("OLS Regression")

    n, k = X.shape

    # note use of Python style zero based indexing
    m.I = pyo.RangeSet(0, n - 1)
    m.J = pyo.RangeSet(0, k - 1)

    m.e = pyo.Var(m.I, domain=pyo.Reals)
    m.m = pyo.Var(m.J)
    m.b = pyo.Var()

    @m.Constraint(m.I)
    def residuals(m, i):
        return m.e[i] == y[i] - sum(X[i][j] * m.m[j] for j in m.J) - m.b

    @m.Objective(sense=pyo.minimize)
    def sum_of_square_errors(m):
        return sum((m.e[i]) ** 2 for i in m.I)

    return m


m = ols_regression(X, y)
SOLVER.solve(m)
m.m.display()
m.b.display()

m : Size=2, Index=J
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
      0 :  None : -0.031936728102144714 :  None : False : False :  Reals
      1 :  None :   0.02093537947512046 :  None : False : False :  Reals
b : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 1.3596789448187763 :  None : False : False :  Reals
